In [ ]:

# ── Branch and Bound Algorithm for Cyclopeptide Sequencing ──
amino_masses = [57, 71, 87, 97, 99, 101, 103, 113, 114, 115,
                128, 129, 131, 137, 147, 156, 163, 186]
mass_to_aa = {
    57:'G', 71:'A', 87:'S', 97:'P', 99:'V', 101:'T', 103:'C',
    113:'L', 114:'N', 115:'D', 128:'Q', 129:'E', 131:'M',
    137:'H', 147:'F', 156:'R', 163:'Y', 186:'W'
}

def linear_spectrum(peptide_masses):
    n = len(peptide_masses)
    prefix = [0]
    for m in peptide_masses:
        prefix.append(prefix[-1] + m)
    spec = [0]
    for i in range(n):
        for j in range(i + 1, n + 1):
            spec.append(prefix[j] - prefix[i])
    return sorted(spec)

def cyclic_spectrum(peptide_masses):
    n = len(peptide_masses)
    prefix = [0]
    for m in peptide_masses:
        prefix.append(prefix[-1] + m)
    total = prefix[-1]
    spec = [0]
    for i in range(n):
        for j in range(i + 1, n + 1):
            mass = prefix[j] - prefix[i]
            spec.append(mass)
            if i > 0 and j < n:          # wrap-around fragments
                spec.append(total - mass)
    return sorted(spec)

def is_consistent(peptide_masses, spectrum):
    lin = linear_spectrum(peptide_masses)
    spec_copy = list(spectrum)
    for m in lin:
        if m in spec_copy:
            spec_copy.remove(m)
        else:
            return False
    return True

def branch_and_bound(spectrum):
    parent_mass = max(spectrum)
    candidates = [[]]          # start with the empty peptide
    results = []

    while candidates:
        new_candidates = []
        for pep in candidates:
            for aa_mass in amino_masses:
                new_candidates.append(pep + [aa_mass])
        candidates = []
        for pep in new_candidates:
            pep_mass = sum(pep)

            if pep_mass == parent_mass:
                if sorted(cyclic_spectrum(pep)) == sorted(spectrum):
                    results.append(pep)
            elif pep_mass < parent_mass:
                if is_consistent(pep, spectrum):
                    candidates.append(pep)
    return results

print("Branch and Bound algorithm defined ")
print(f"Using {len(amino_masses)} unique amino acid masses")


Branch and Bound algorithm defined 
Using 18 unique amino acid masses


In [5]:
example_peptide = [114, 128, 129, 113]   # N-Q-E-L
example_spectrum = cyclic_spectrum(example_peptide)

print("Spectrum:", example_spectrum)
print("Parent mass:", max(example_spectrum))

results = branch_and_bound(example_spectrum)

print(f"\nFound {len(results)} cyclopeptide(s):")
for pep in results:
    print("  ", "-".join(str(m) for m in pep))


Spectrum: [0, 113, 114, 128, 129, 227, 242, 242, 257, 355, 356, 370, 371, 484]
Parent mass: 484

Found 8 cyclopeptide(s):
  113-114-128-129
  113-129-128-114
  114-113-129-128
  114-128-129-113
  128-114-113-129
  128-129-113-114
  129-113-114-128
  129-128-114-113
